In [29]:
from bitcoinutils.keys import P2trAddress, PrivateKey
from bitcoinutils.script import Script
from bitcoinutils.utils import ControlBlock
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.setup import setup
import json
import codecs
import binascii


def set_network(network: str = "testnet"):
  setup(network)

def coonvert_string_to_hex(content: str):
  """
  将字符串转化为hex
  """
  bt = codecs.encode(content, 'utf-8')
  hex_string = binascii.hexlify(bt).decode('utf-8')
  return hex_string

def generate_random_private_key():
  """
  生成随机私钥
  """
  priv = PrivateKey()
  return priv.to_bytes().hex()

import json

def generate_brc20_mint_inscribe_content(tick: str, amt: int):
  content = {
    "p": "brc-20",
    "op": "mint",
    "tick": tick,
    "amt": str(amt),
  }
  return json.dumps(content)

def generate_brc20_deploy_inscribe_content(tick: str, max_supply: int, limit: int):
  content = {
    "p": "brc-20",
    "op": "deploy",
    "tick": tick,
    "max": str(max_supply),
    "lim": str(limit),
  }
  return json.dumps(content)


def get_bitmap_script_pub_key(private: str, block_nummber: int):
  """
  生成铭文的scriptPubKey
  """
  # 重复但为了解耦
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()
  tr_script_p2pk = Script([
    pubkey_tr_script.to_x_only_hex(),
    "OP_CHECKSIG",
    "OP_0",
    "OP_IF",
    coonvert_string_to_hex("ord"),
    "01",
    coonvert_string_to_hex("text/plain;charset=utf-8"),
    "OP_0",
    coonvert_string_to_hex(str(block_nummber) + ".bitmap"),
    "OP_ENDIF",
  ])
  return tr_script_p2pk

def get_bitmap_inscription_address(private: str, block_nummber: int):
  """
  生成铭文的转入地址
  """
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()
  tr_script_p2pk = get_bitmap_script_pub_key(private, block_nummber)

  # taproot script path address
  from_address = pubkey_tr_script.get_taproot_address([[tr_script_p2pk]])
  print("From Taproot script address", from_address.to_string())
  return from_address.to_string()

def inscribe_after_tx(txid: str, vout: int, amount: int, private: str, block_nummber: int, receive_address: str):
  """
  铭刻交易
  """
  # taproot script is a simple P2PK with the following keys
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()
  tr_script_p2pk = get_bitmap_script_pub_key(private, block_nummber)

  # taproot script path address
  from_address = pubkey_tr_script.get_taproot_address([[tr_script_p2pk]])
  print("From Taproot script address", from_address.to_string())

  # create transaction input from tx id of UTXO
  tx_in = TxInput(txid, vout)

  amounts = [amount]

  scriptPubkey = from_address.to_script_pub_key()
  utxos_scriptPubkeys = [scriptPubkey]
  toAddress = P2trAddress(receive_address)

  tx_out = TxOutput(546, toAddress.to_script_pub_key())
  tx = Transaction([tx_in], [tx_out], has_segwit=True)

  sig = privkey_tr_script.sign_taproot_input(
      tx,
      0,
      utxos_scriptPubkeys,
      amounts,
      script_path=True,
      tapleaf_script=tr_script_p2pk,
      tweak=False,
  )

  # we spend a single script - no merkle path is required
  control_block = ControlBlock(pubkey_tr_script)

  tx.witnesses.append(
      TxWitnessInput([sig, tr_script_p2pk.to_hex(), control_block.to_hex()])
  )
  # print raw signed transaction ready to be broadcasted
  return [tx.serialize(), tx.get_vsize(), tx.get_txid()]

def inscribe_and_send_after_tx(
    txid: str,
    vout: int,
    amount: int,
    fee: int,
    private: str,
    block_nummber: int,
    receive_address: str,
    send_address: str,
  ):
  # taproot script is a simple P2PK with the following keys
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()
  tr_script_p2pk = get_bitmap_script_pub_key(private, block_nummber)

  # taproot script path address
  from_address = pubkey_tr_script.get_taproot_address([[tr_script_p2pk]])
  print("From Taproot script address", from_address.to_string())

  # create transaction input from tx id of UTXO
  tx_in = TxInput(txid, vout)

  amounts = [amount]

  scriptPubkey = from_address.to_script_pub_key()
  utxos_scriptPubkeys = [scriptPubkey]
  toAddress = P2trAddress(receive_address)
  refundAddress = P2trAddress(send_address)

  base = 546
  tx_out_list = []
  if amount - base > 0:
    tx_out_inscription = TxOutput(base, toAddress.to_script_pub_key())
    tx_out_list.append(tx_out_inscription)
  else:
    tx_out_inscription = TxOutput(amount, toAddress.to_script_pub_key())
    tx_out_list.append(tx_out_inscription)
  if amount - base - fee > 0:
    tx_out_refund = TxOutput(amount - fee - base, refundAddress.to_script_pub_key())
    tx_out_list.append(tx_out_refund)

  print(tx_out_list)
  
  tx = Transaction([tx_in], tx_out_list, has_segwit=True)
  # sign taproot input
  # to create the digest message to sign in taproot we need to
  # pass all the utxos' scriptPubKeys, their amounts and taproot script
  # we sign with the private key corresponding to the script - no keys
  # tweaking required
  sig = privkey_tr_script.sign_taproot_input(
      tx,
      0,
      utxos_scriptPubkeys,
      amounts,
      script_path=True,
      tapleaf_script=tr_script_p2pk,
      tweak=False,
  )

  # we spend a single script - no merkle path is required
  control_block = ControlBlock(pubkey_tr_script)

  tx.witnesses.append(
      TxWitnessInput([sig, tr_script_p2pk.to_hex(), control_block.to_hex()])
  )
  # print raw signed transaction ready to be broadcasted
  return [tx.serialize(), tx.get_vsize(), tx.get_txid()]

def refund_from_inscription_tx(
    txid: str,
    vout: int,
    amount: int,
    fee: int,
    private: str,
    block_nummber: int,
    receive_address: str,
):
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()
  tr_script_p2pk = get_bitmap_script_pub_key(private, block_nummber)

  # taproot script path address
  from_address = pubkey_tr_script.get_taproot_address([[tr_script_p2pk]])

  # create transaction input from tx id of UTXO
  tx_in = TxInput(txid, vout)

  amounts = [amount]

  scriptPubkey = from_address.to_script_pub_key()
  utxos_scriptPubkeys = [scriptPubkey]
  toAddress = P2trAddress(receive_address)

  tx_out_list = []
  tx_out_inscription = TxOutput(amount - fee, toAddress.to_script_pub_key())
  tx_out_list.append(tx_out_inscription)

  print(tx_out_list)
  
  tx = Transaction([tx_in], tx_out_list, has_segwit=True)


  sig = privkey_tr_script.sign_taproot_input(
        tx,
        0,
        utxos_scriptPubkeys,
        amounts,
        script_path=False,
        tapleaf_scripts=[tr_script_p2pk],
    )

  tx.witnesses.append(TxWitnessInput([sig]))
  # print raw signed transaction ready to be broadcasted
  return [tx.serialize(), tx.get_vsize(), tx.get_txid()]




In [30]:
set_network('testnet')

In [3]:
priv = generate_random_private_key()
print("priv", priv)

priv aa33408688ed3b5c3e66b44d0a65aa21fdf9d2b845bec51e58de593819a5766e


In [17]:


def get_text_script_pub_key(private: str, content: str):
  """
  生成铭文的scriptPubKey
  """
  # 重复但为了解耦
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()
  tr_script_p2pk = Script([
    pubkey_tr_script.to_x_only_hex(),
    "OP_CHECKSIG",
    "OP_0",
    "OP_IF",
    coonvert_string_to_hex("ord"),
    "01",
    coonvert_string_to_hex("text/plain;charset=utf-8"),
    "OP_0",
    coonvert_string_to_hex(content),
    "OP_ENDIF",
  ])
  return tr_script_p2pk

script_pubkey = get_text_script_pub_key(priv, generate_brc20_mint_inscribe_content("mice", 1000))
print(script_pubkey)

['c75b8966a90d0d7bd16cbb3d0222cf682a4aaff546f3196fab1375d77cdd14a9', 'OP_CHECKSIG', 'OP_0', 'OP_IF', '6f7264', '01', '746578742f706c61696e3b636861727365743d7574662d38', 'OP_0', '7b2270223a20226272632d3230222c20226f70223a20226d696e74222c20227469636b223a20226d696365222c2022616d74223a202231303030227d', 'OP_ENDIF']


In [52]:
def get_inscription_address(private: str, tr_script_p2pk: Script):
  """
  生成铭文的转入地址
  """
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()

  # taproot script path address
  from_address = pubkey_tr_script.get_taproot_address([[tr_script_p2pk]])
  print("From Taproot script address", from_address.to_string())
  return from_address.to_string()

address = get_inscription_address(priv, get_text_script_pub_key(priv, generate_brc20_deploy_inscribe_content("mice", 210000000, 1000)))
print("address", address)

From Taproot script address tb1pahlzgcvd00etd4vszgqvd9rzwcqdpfsa23y7d7mq4l3472m6xtjq63ty6e
address tb1pahlzgcvd00etd4vszgqvd9rzwcqdpfsa23y7d7mq4l3472m6xtjq63ty6e


In [48]:
address = get_brc20_deploy_inscription_address(priv, "mice", 210000000, 1000)
print("address", address)

From Taproot script address tb1pahlzgcvd00etd4vszgqvd9rzwcqdpfsa23y7d7mq4l3472m6xtjq63ty6e
address tb1pahlzgcvd00etd4vszgqvd9rzwcqdpfsa23y7d7mq4l3472m6xtjq63ty6e


In [19]:

def get_brc20_mint_inscription_address(private: str, tick: str, amt: int):
  """
  生成铭文的转入地址
  """
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()
  tr_script_p2pk = get_text_script_pub_key(priv, generate_brc20_inscribe_content(tick, amt))

  # taproot script path address
  from_address = pubkey_tr_script.get_taproot_address([[tr_script_p2pk]])
  print("From Taproot script address", from_address.to_string())
  return from_address.to_string()

address = get_brc20_mint_inscription_address(priv, "mice", 1000)
print("address", address)

From Taproot script address tb1prg5keay3tu7vkwj00kvjxz84flxrvfa4k9jj9tppks7at27yx0tqjjyexe
address tb1prg5keay3tu7vkwj00kvjxz84flxrvfa4k9jj9tppks7at27yx0tqjjyexe


In [33]:
def inscribe_after_tx(txid: str, vout: int, amount: int, private: str, tr_script_p2pk: Script, receive_address: str):
  """
  铭刻交易
  """
  # taproot script is a simple P2PK with the following keys
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()

  # taproot script path address
  from_address = pubkey_tr_script.get_taproot_address([[tr_script_p2pk]])
  print("From Taproot script address", from_address.to_string())

  # create transaction input from tx id of UTXO
  tx_in = TxInput(txid, vout)

  amounts = [amount]

  scriptPubkey = from_address.to_script_pub_key()
  utxos_scriptPubkeys = [scriptPubkey]
  toAddress = P2trAddress(receive_address)

  tx_out = TxOutput(546, toAddress.to_script_pub_key())
  tx = Transaction([tx_in], [tx_out], has_segwit=True)

  sig = privkey_tr_script.sign_taproot_input(
      tx,
      0,
      utxos_scriptPubkeys,
      amounts,
      script_path=True,
      tapleaf_script=tr_script_p2pk,
      tweak=False,
  )

  # we spend a single script - no merkle path is required
  control_block = ControlBlock(pubkey_tr_script)

  tx.witnesses.append(
      TxWitnessInput([sig, tr_script_p2pk.to_hex(), control_block.to_hex()])
  )
  # print raw signed transaction ready to be broadcasted
  return [tx.serialize(), tx.get_vsize(), tx.get_txid()]

In [22]:

[txhex, vsize, txid] = inscribe_after_tx('4f1c1ffe9135f95027852eb766140bbfdb76051f202de58404504e09340f31e1', 0, 2000, priv, "mice", 1000, "tb1pfywwjgdmskq4gjc55mehzt8u6a29mjslwl5avdphfyvs0grtgwrs4c768e")

From Taproot script address tb1prg5keay3tu7vkwj00kvjxz84flxrvfa4k9jj9tppks7at27yx0tqjjyexe


In [25]:
import requests

def broadcast_tx_v2(tx_hex, network="testnet"):
    """
        广播交易到链上, 使用mempool
    """
    net = ""
    if network == "testnet":
        net = "testnet/"
    # 设置基本的 URL
    base_url = "https://mempool.space/" + net + "api/tx"

    try:
        response = requests.post(base_url, tx_hex)
        print("response:", response)
        return response
    except Exception as e:
        print("call blockchain api to broadcast tx error, message:", e)
        return None

In [27]:
response = broadcast_tx_v2(txhex)

response: <Response [200]>


In [28]:
response.text

'113acd85bd18a32a2f6c6d0570611bd27115b2bd04c6a0ac28d8bc17f74a1fbf'

In [56]:
tx = {
  "txid": "e2edd441b84db52987d487d0f2bcb2aa3957264e77f63aaeb26de7a1b8327d61",
  "vout": 0,
  "amount": 1000,
}

[txhex, vsize, txid] = inscribe_after_tx(
  tx['txid'],
  tx['vout'],
  tx['amount'],
  priv,
  get_text_script_pub_key(priv, generate_brc20_deploy_inscribe_content("mice", 210000000, 1000)),
  "tb1pfywwjgdmskq4gjc55mehzt8u6a29mjslwl5avdphfyvs0grtgwrs4c768e"
)
response = broadcast_tx_v2(txhex)
'tb1p3wxap6l3wpfktd05agcsvu3trwpmd2dz67urhwydk6dtke2c0fdq9lu9lz'
print(response.text)

From Taproot script address tb1pahlzgcvd00etd4vszgqvd9rzwcqdpfsa23y7d7mq4l3472m6xtjq63ty6e
response: <Response [200]>
333e74c5c906437ba8f299c6e45240fc9b06a8f95211bfc282014d950853ccff


In [58]:
tx = {
  "txid": "34ceb094d6af037e4dd2b2142377d5e05a6af05379203faecffd23653286ac6a",
  "vout": 0,
  "amount": 1000,
}

[txhex, vsize, txid] = inscribe_after_tx(
  tx['txid'],
  tx['vout'],
  tx['amount'],
  priv,
  get_text_script_pub_key(priv, generate_brc20_mint_inscribe_content("mice", 1000)),
  "tb1pfywwjgdmskq4gjc55mehzt8u6a29mjslwl5avdphfyvs0grtgwrs4c768e"
)
response = broadcast_tx_v2(txhex)
'tb1p3wxap6l3wpfktd05agcsvu3trwpmd2dz67urhwydk6dtke2c0fdq9lu9lz'
print(response.text)

From Taproot script address tb1prg5keay3tu7vkwj00kvjxz84flxrvfa4k9jj9tppks7at27yx0tqjjyexe
response: <Response [200]>
a4199b90ca68d32aa759ed30e2324e003863707b03f3aa5202b18c13060d36e3


In [35]:
def refund_from_inscription_tx(
    txid: str,
    vout: int,
    amount: int,
    fee: int,
    private: str,
    tr_script_p2pk: Script,
    receive_address: str,
):
  privkey_tr_script = PrivateKey.from_bytes(bytes.fromhex(private))
  pubkey_tr_script = privkey_tr_script.get_public_key()

  # taproot script path address
  from_address = pubkey_tr_script.get_taproot_address([[tr_script_p2pk]])

  # create transaction input from tx id of UTXO
  tx_in = TxInput(txid, vout)

  amounts = [amount]

  scriptPubkey = from_address.to_script_pub_key()
  utxos_scriptPubkeys = [scriptPubkey]
  toAddress = P2trAddress(receive_address)

  tx_out_list = []
  tx_out_inscription = TxOutput(amount - fee, toAddress.to_script_pub_key())
  tx_out_list.append(tx_out_inscription)

  print(tx_out_list)
  
  tx = Transaction([tx_in], tx_out_list, has_segwit=True)


  sig = privkey_tr_script.sign_taproot_input(
        tx,
        0,
        utxos_scriptPubkeys,
        amounts,
        script_path=False,
        tapleaf_scripts=[tr_script_p2pk],
    )

  tx.witnesses.append(TxWitnessInput([sig]))
  # print raw signed transaction ready to be broadcasted
  return [tx.serialize(), tx.get_vsize(), tx.get_txid()]

[txhex, vsize, txid] = refund_from_inscription_tx(
  "cd783cf64b209e4bb5952148db6b17d25ff77c7f4f368dd5443afc752a279cb6",
  0,
  1000,
  141,
  priv,
  get_text_script_pub_key(priv, generate_brc20_mint_inscribe_content("mice", 1000)),
  "tb1pacxdwxtyu3h48wg2g37hd8g2l7nnrekpklwgxv4glnvxsl876jdqa4chp3"
)

response = broadcast_tx_v2(txhex)
print(response.text)


[{'amount': 859, 'script_pubkey': ['OP_1', 'ee0cd71964e46f53b90a447d769d0affa731e6c1b7dc8332a8fcd8687cfed49a']}]
response: <Response [200]>
a369c441dbd8fdcdc0898c23a84987191d5d4755ca37c7d6ce7b964f18a10325


In [44]:
tx = {
  "txid": "a64663743a0932aa30ccb49613881d5f59ac8734e8780ee3fcf73126e2645e63",
  "vout": 0,
  "amount": 1600,
}

[txhex, vsize, txid] = refund_from_inscription_tx(
  tx["txid"],
  tx["vout"],
  tx["amount"],
  141,
  priv,
  get_text_script_pub_key(priv, generate_brc20_deploy_inscribe_content("mice", 210000000, 1000)),
  "tb1pacxdwxtyu3h48wg2g37hd8g2l7nnrekpklwgxv4glnvxsl876jdqa4chp3"
)

response = broadcast_tx_v2(txhex)
print(response.text)

[{'amount': 1459, 'script_pubkey': ['OP_1', 'ee0cd71964e46f53b90a447d769d0affa731e6c1b7dc8332a8fcd8687cfed49a']}]
response: <Response [200]>
7485c872c1dbe75d024865a98d1cb9617993415a58b7efdef02dc73a11e19da9


In [1]:
import os
import sys

# 引入本地模块
# 获得父目录路径
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))

In [2]:
from client.blockchain_client import address_received_money_in_this_tx

result = await address_received_money_in_this_tx("tb1p52ex09p6en3czr0rs7sn5q2zw5t6ueasxtvnfshfe2ef2h2x5khqh2h5qm")
print(result)

[{'txid': '7ca44b3ace26c2894e84cbb4985d8bd7105f27eb587e540e30cd0844fd6e5244', 'version': 2, 'locktime': 0, 'vin': [{'txid': '9edd046a9a915b8764724acb2454ec57431a480a3d86ec28b80a5a5ffe52f11a', 'vout': 1, 'prevout': {'scriptpubkey': '5120ee0cd71964e46f53b90a447d769d0affa731e6c1b7dc8332a8fcd8687cfed49a', 'scriptpubkey_asm': 'OP_PUSHNUM_1 OP_PUSHBYTES_32 ee0cd71964e46f53b90a447d769d0affa731e6c1b7dc8332a8fcd8687cfed49a', 'scriptpubkey_type': 'v1_p2tr', 'scriptpubkey_address': 'tb1pacxdwxtyu3h48wg2g37hd8g2l7nnrekpklwgxv4glnvxsl876jdqa4chp3', 'value': 627}, 'scriptsig': '', 'scriptsig_asm': '', 'witness': ['e1bc420ba0ec4a362fb1343c9058a46bcb16efc34b5b424755dbe1a2cf9f21061fbb654b5cb6c6df060ae80128bca3bda5f1893f884d5acaade91bfde6030aef'], 'is_coinbase': False, 'sequence': 4294967295}, {'txid': 'a64663743a0932aa30ccb49613881d5f59ac8734e8780ee3fcf73126e2645e63', 'vout': 1, 'prevout': {'scriptpubkey': '5120ee0cd71964e46f53b90a447d769d0affa731e6c1b7dc8332a8fcd8687cfed49a', 'scriptpubkey_asm': 'OP_P

In [2]:
from client.blockchain_client import address_once_had_money

result = await address_once_had_money("tb1p52ex09p6en3czr0rs7sn5q2zw5t6ueasxtvnfshfe2ef2h2x5khqh2h5qm", network="testnet")
print(result)

https://mempool.space/testnet/api/address/tb1p52ex09p6en3czr0rs7sn5q2zw5t6ueasxtvnfshfe2ef2h2x5khqh2h5qm
response: {"address":"tb1p52ex09p6en3czr0rs7sn5q2zw5t6ueasxtvnfshfe2ef2h2x5khqh2h5qm","chain_stats":{"funded_txo_count":1,"funded_txo_sum":1000,"spent_txo_count":0,"spent_txo_sum":0,"tx_count":1},"mempool_stats":{"funded_txo_count":0,"funded_txo_sum":0,"spent_txo_count":0,"spent_txo_sum":0,"tx_count":0}}
True
